In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/Short Films/Shortfilm Data/Shortfilms HandTagged/Shortfilms - InteriorCafeNight.csv') # this is just a example change the file name and location as needed


# BGM Analysis

In [ ]:
!git clone https://github.com/purnima99/EmotionDetection.git


In [ ]:
%cd EmotionDetection/Emotion-Detection-from-Audio


In [ ]:
!pip install tensorflow
!pip install librosa


In [ ]:
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model


In [ ]:
from tensorflow.keras.models import load_model

model_path = '/content/EmotionDetection/Emotion-Detection-from-Audio/model/Emotion_Voice_Detection_Model.h5'
model = load_model(model_path)


In [ ]:
import pandas as pd

def duration_to_seconds(duration):
    # Split the duration into hours, minutes, and seconds
    hours, minutes, seconds = map(int, duration.split(':'))

    # Convert the duration to seconds
    total_seconds = (hours * 3600) + (minutes * 60) + seconds
    return total_seconds

In [ ]:
def extract_features(audio):
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=40).T, axis=0)
    return mfccs

def predict_emotion_for_time_frame(mp3_file_path, start_time, end_time):
    # Load the long MP3 file
    audio, sr = librosa.load(mp3_file_path, sr=22050)

    # Calculate the start and end sample indices
    start_sample = int(start_time * sr)
    end_sample = int(end_time * sr)

    # Extract the audio segment for the specified time frame
    audio_segment = audio[start_sample:end_sample]

    # Extract features from the audio segment
    features = extract_features(audio_segment)
    features = np.expand_dims(features, axis=0)
    features = np.expand_dims(features, axis=2)

    # Perform emotion prediction
    predictions = model.predict(features)
    predicted_class = np.argmax(predictions)
    label_conversion = {'0': 'neutral','1': 'calm','2': 'happy','3': 'sad','4': 'angry','5': 'fearful','6': 'disgust','7': 'surprised'}
    for key, value in label_conversion.items():
      if int(key) == predicted_class:
        predicted_emotion = value

    return predicted_emotion


In [ ]:
mp3_file_path = '/content/Short Films/BGM/Aai Shapat.mp3' #changed the file path as per need

In [ ]:
def emoBGM(df, mp3_file_path):
  # Create a new column for predicted emotion
  df['PredictedEmotionBGM'] = ''

  # Iterate over each row in the DataFrame
  for index, row in df.iterrows():
      # Extract the start time and end time for the row
      start_time = row['StartTime']
      end_time = row['EndTime']

      # Predict the emotion for the time frame
      emotion = predict_emotion_for_time_frame(mp3_file_path, start_time, end_time)

      # Store the predicted emotion in the 'PredictedEmotion' column
      df.at[index, 'PredictedEmotionBGM'] = emotion


Running the model on the DataFrame

In [ ]:
df['StartTime'] = df['StartTime'].apply(duration_to_seconds)
df['EndTime'] = df['EndTime'].apply(duration_to_seconds)


In [ ]:
df = emoBGM(df)

In [ ]:
# Save the DataFrame as a CSV file
df.to_csv('InteriorCafeNight.csv', index=False)